# BentoML Example: Fast AI with Tabular data

This notebook is based on fastai's cours v3 lesson 4.  We are going to train a model that predict salary range base on the data we provided.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. In this project we will use BentoML to package the trained fast.ai model, and build a containerized REST API model server.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai
!pip install bentoml

In [2]:
from fastai.tabular import *

## Prepare Training Data

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [4]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [5]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [6]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [7]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,False,0.8365,1.6025,-0.4224,<50k
Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,False,0.0303,1.2028,1.5334,<50k
Private,12th,Never-married,Transport-moving,Not-in-family,White,False,-0.8493,-1.4328,-0.8135,<50k
Private,Some-college,Married-civ-spouse,Other-service,Wife,White,False,-1.0692,-1.4924,-0.0312,<50k
Federal-gov,11th,Divorced,Craft-repair,Not-in-family,Black,False,1.4229,-1.1066,-1.2046,>=50k
Private,HS-grad,Married-civ-spouse,Sales,Husband,White,False,0.9831,-0.5826,-0.4224,<50k
Private,10th,Never-married,Machine-op-inspct,Not-in-family,White,False,-1.3624,2.7780,-1.5958,<50k
Private,Bachelors,Never-married,Sales,Other-relative,Black,False,-1.3624,0.2533,1.1422,<50k
Private,Doctorate,Divorced,Sales,Not-in-family,White,False,-0.4095,-0.1797,2.3157,>=50k
Private,11th,Married-civ-spouse,Transport-moving,Husband,White,False,-1.1425,-0.0124,-1.2046,<50k


## Model Training

In [8]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [9]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.369908,0.387708,0.830000,00:02


In [10]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category >=50k, tensor(1), tensor([0.4603, 0.5397]))

## Create BentoService for model serving

In [11]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import DataframeHandler


@env(pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('model')])
class TabularModel(BentoService):
    
    @api(DataframeHandler)
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Overwriting tabular_csv.py


## Save BentoService to file archive

In [12]:
# 1) import the custom BentoService defined above
from tabular_csv import TabularModel

# 2) `pack` it with required artifacts
svc = TabularModel.pack(model=learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2019-10-14 17:23:22,673] INFO - Successfully saved Bento 'TabularModel:20191014172300_C209CA' to path: /Users/chaoyuyang/bentoml/repository/TabularModel/20191014172300_C209CA


## Install saved BentoService as PyPI package

In [13]:
!pip install {saved_path}

Processing /Users/chaoyuyang/bentoml/repository/TabularModel/20191014172300_C209CA
  Stored in directory: /private/var/folders/ns/vc9qhmqx5dx_9fws7d869lqh0000gn/T/pip-ephem-wheel-cache-2hpija8j/wheels/55/28/15/ab43b4238057682b6c3eb85c7cafd148b1b168409e552fcbb7
Successfully built TabularModel
  Found existing installation: TabularModel 20191014171509-6B8DEF
    Uninstalling TabularModel-20191014171509-6B8DEF:
      Successfully uninstalled TabularModel-20191014171509-6B8DEF


In [14]:
# Use json data
!TabularModel predict --input=test.json

['<50k']


In [15]:
# Use CSV data
!TabularModel predict --input=test.csv

['>=50k']


## Model Serving via REST API

*Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally*

In [ ]:
!bentoml serve {saved_path}

 * Serving Flask app "TabularModel" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Oct/2019 17:26:37] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2019 17:26:39] "POST /predict HTTP/1.1" 200 -


### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```